In [2]:
#DB Connection 
import pymssql
import pandas as pd
import pyodbc
import numpy as np
conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                     "Server=Martin-PC\SQLEXPRESS;"
                     "Database=FootballData;"
                     "Trusted_Connection=yes;")
#conn = pymssql.connect(server="localhost", user="",password="", port=63642)  # You can lookup the port number inside SQL server. 

data_folder = 'data/'

In [3]:
#Matches select
stmt = """SELECT Teams.FullName as HomeTeam, Teams2.FullName as AwayTeam, 
        Teams.ExternalId as HomeTeamId, Teams2.ExternalId as AwayTeamId
	  ,[Matches].ExternalId as ExternalId
      ,[Date]
      ,[Country]
      ,[League]
      ,[Season]
      ,[Stage]
      ,[AwayTeam_Id]
      ,[HomeTeam_Id]
FROM  [FootballData].[dbo].[Matches]
LEFT JOIN [Teams] ON Matches.HomeTeam_Id = Teams.Id
LEFT JOIN [Teams] as Teams2 ON Matches.AwayTeam_Id = Teams2.Id"""
# Excute Query here
df_matches = pd.read_sql(stmt,conn)
df_matches.drop_duplicates(['ExternalId'], inplace=True)
df_matches['Date']=pd.to_datetime(df_matches['Date'])
print(df_matches.shape)
df_matches.head(3)

(7750, 12)


,HomeTeam,AwayTeam,HomeTeamId,AwayTeamId,ExternalId,Date,Country,League,Season,Stage,AwayTeam_Id,HomeTeam_Id
0,Leicester,Everton,8197,8668,1723984,2014-08-18 15:00:00,England,Premier League,2014/2015,1,13,9
1,Burnley,Chelsea,8191,8455,1723983,2014-08-18 15:00:00,England,Premier League,2014/2015,1,14,10
2,Liverpool,Southampton,8650,8466,1723985,2014-08-18 15:00:00,England,Premier League,2014/2015,1,15,12


In [4]:
df_matches['Date'].min()

Timestamp('2014-08-10 15:00:00')

In [5]:
df_matches['Date'].max()

Timestamp('2019-05-12 14:00:00')

In [6]:
#Shots on select
stmt = "SELECT * FROM ShotOns"
# Excute Query here
df_shots_on = pd.read_sql(stmt,conn)
#df_corners = df_corners['Minute'].notnull()
print(df_shots_on.shape)
df_shots_on.drop_duplicates(['ExternalId'], inplace=True)

#Shots off select
stmt = "SELECT * FROM ShotOffs"
# Excute Query here
df_shots_off = pd.read_sql(stmt,conn)
#df_corners = df_corners['Minute'].notnull()
print(df_shots_off.shape)
df_shots_off.drop_duplicates(['ExternalId'], inplace=True)

#Possessions select
stmt = "SELECT * FROM Possessions"
# Excute Query here
df_possessions = pd.read_sql(stmt,conn)
#df_corners = df_corners['Minute'].notnull()
print(df_possessions.shape)
df_possessions.drop_duplicates(['ExternalId'], inplace=True)
df_possessions.replace('', np.nan, inplace=True)
df_possessions.dropna(axis=0, how='any', inplace=True)

df_possessions['HomePossession'] = df_possessions['HomePossession'].astype(int)
df_possessions['AwayPossession'] = df_possessions['AwayPossession'].astype(int)
df_possessions['Minute'] = df_possessions['Minute'].astype(int)
#df_possessions['TeamId'] = df_possessions['TeamId'].astype(str)
df_possessions.sample(5)


(83999, 6)
(82698, 6)
(104795, 7)


,Id,ExternalId,MatchId,Minute,HomePossession,AwayPossession,TeamId
104309,104310,8535572,2782029,70,49,51,0
24325,24326,6068295,2241434,30,52,48,0
100172,100173,9457971,2782261,30,56,44,0
61947,61948,6651755,2319992,50,58,42,0
43507,43508,6321812,2272453,55,37,63,0


In [7]:
#Goals select
stmt = "SELECT * FROM Goals"
# Excute Query here
df_goals = pd.read_sql(stmt,conn)
print(df_goals.shape)
df_goals.drop('Id', inplace=True, axis=1)
df_goals.drop_duplicates(inplace=True)

(22244, 8)


In [8]:
#Corners select
stmt = "SELECT * FROM Corners"
# Excute Query here
df_corners = pd.read_sql(stmt,conn)
#df_corners = df_corners['Minute'].notnull()
print(df_corners.shape)
df_corners.drop_duplicates(['ExternalId'], inplace=True)
df_corners.sample(3)

(78204, 6)


,Id,ExternalId,MatchId,Minute,Player1,TeamId
36574,36575,4769043,2002122,56,34267,8226
61064,61065,4334987,1778326,41,279173,10268
68515,68516,6532613,2300794,88,26181,8302


In [9]:
#Teams select
stmt = "SELECT DISTINCT ExternalId FROM Teams"
# Excute Query here
df_teams = pd.read_sql(stmt,conn)
print(df_teams.shape)
df_teams.head(2)
#df_teams.count

(135, 1)


,ExternalId
0,10242
1,9941


In [10]:
# Group by corners teams matches not working
# stmt = """SELECT  Corners.TeamId, convert (bigint, minute  ) as Minute, count(*) as count
#   FROM [FootballData].[dbo].[Corners] 
#   LEFT JOIN [Teams] ON Corners.TeamId = Teams.ExternalId
#   LEFT JOIN [Matches] ON MatchId = Corners.MatchId
#   group by convert (bigint, minute  ),TeamId
#   order by 1,2"""
# # Excute Query here

#Corners select
stmt = """SELECT DISTINCT  Matches.Id as MatchId, Corners.ExternalId, convert (bigint,  Corners.Minute) as Minute, Teams.ExternalId as TeamId,
    Teams.FullName, Teams.Acronym,Matches.Season
 FROM [FootballData].[dbo].[Teams]
RIGHT JOIN [Corners] ON Corners.TeamId = Teams.ExternalId
LEFT JOIN [Matches] ON [Matches].ExternalId = Corners.MatchId

AND Matches.Season = '2018/2019'
"""
#WHERE League like 'Premier League'
df_corners_per_team = pd.read_sql(stmt,conn)
#df_corners = df_corners['Minute'].notnull()

df_corners_per_team.drop_duplicates(['ExternalId'], inplace=True)
print(df_corners_per_team.shape)
df_corners_per_team.sample(3)

(77384, 7)


,MatchId,ExternalId,Minute,TeamId,FullName,Acronym,Season
68831,NaN,7427146,83,8177,Hertha Berlin,HBE,None
67789,NaN,7386931,53,9851,Rennes,REN,None
85313,7655.0,9136866,61,10261,Newcastle United,NCU,2018/2019


In [11]:
stmt = """SELECT DISTINCT  Matches.EXTERNALID as MatchId, Corners.MatchId as CornerMatchId, convert (bigint,  Corners.Minute) as Minute, Teams.ExternalId as TeamId,
    Teams.FullName,Matches.Season
From MAtches 
 LEFT Join 
	(SELECT MatchId, TeamId, Min(Minute) as Minute
            FROM Corners
            GROUP BY MatchId, TeamId) Corners
ON Corners.MatchID= MAtches.eXTERNALId
LEFT JOIN [FootballData].[dbo].[Teams]
on Matches.HomeTeam_Id = Teams.Id OR Matches.AwayTeam_Id = Teams.ID
"""
#WHERE CORNERS.MATCHID IS NULL 
#WHERE League like 'Premier League'

df_first_corner = pd.read_sql(stmt,conn)
#df_corners = df_corners['Minute'].notnull()

df_first_corner.head(5)

,MatchId,CornerMatchId,Minute,TeamId,FullName,Season
0,1709697,1709697.0,15.0,9837,Reims,2014/2015
1,1709697,1709697.0,15.0,9847,Paris Saint Germain,2014/2015
2,1709697,1709697.0,52.0,9837,Reims,2014/2015
3,1709697,1709697.0,52.0,9847,Paris Saint Germain,2014/2015
4,1709698,1709698.0,20.0,7794,SC Bastia,2014/2015


In [12]:
stmt = """
SELECT DISTINCT  Matches.EXTERNALID as MatchId, Goals.MatchId as CornerMatchId, convert (bigint,  Goals.Minute) as Minute, Teams.ExternalId as TeamId, Teams2.FullName,
    Teams.FullName,Matches.Season
From MAtches 
 left Join 
	(SELECT MatchId, TeamId, Min(Minute) as Minute
            FROM Goals
            GROUP BY MatchId, TeamId) Goals
ON Goals.MatchID= MAtches.eXTERNALId
LEFT JOIN [Teams] ON Matches.HomeTeam_Id = Teams.Id
LEFT JOIN [Teams] as Teams2 ON Matches.AwayTeam_Id = Teams2.Id
"""
#WHERE CORNERS.MATCHID IS NULL 
#WHERE League like 'Premier League'

df_first_goal = pd.read_sql(stmt,conn)
#df_corners = df_corners['Minute'].notnull()

df_first_goal.head(20)

,MatchId,CornerMatchId,Minute,TeamId,FullName,FullName,Season
0,1709697,1709697.0,19.0,9837,Paris Saint Germain,Reims,2014/2015
1,1709697,1709697.0,22.0,9837,Paris Saint Germain,Reims,2014/2015
2,1709698,1709698.0,12.0,7794,Marseille,SC Bastia,2014/2015
3,1709698,1709698.0,17.0,7794,Marseille,SC Bastia,2014/2015
4,1709699,1709699.0,12.0,4087,Caen,Evian Thonon Gaillard,2014/2015
5,1709700,1709700.0,39.0,9747,Saint-Etienne,Guingamp,2014/2015
6,1709701,NaN,NaN,8639,Metz,Lille,2014/2015
7,1709702,1709702.0,64.0,9748,Rennes,Lyon,2014/2015
8,1709703,1709703.0,10.0,9829,Lorient,Monaco,2014/2015
9,1709703,1709703.0,78.0,9829,Lorient,Monaco,2014/2015


In [13]:
stmt = """SELECT EXTERNALID FROM MATCHES 
EXCEPT 
SELECT MATCHID FROM CORNERS
"""
#WHERE CORNERS.MATCHID IS NULL 
#WHERE League like 'Premier League'

df_matches_no_corner = pd.read_sql(stmt,conn)
#df_corners = df_corners['Minute'].notnull()
df_matches_no_corner.shape

(6, 1)

In [14]:
stmt = """SELECT EXTERNALID FROM MATCHES 
EXCEPT 
SELECT MATCHID FROM GOALS
"""
#WHERE CORNERS.MATCHID IS NULL 
#WHERE League like 'Premier League'

df_matches_no_goals = pd.read_sql(stmt,conn)
#df_corners = df_corners['Minute'].notnull()
df_matches_no_goals.shape

(513, 1)

In [15]:
#df_teams_corners_per_minute['Season'] == '2016/2017'
#df_teams_corners_per_minute = df_teams_corners_per_minute[(df_teams_corners_per_minute['League'] == 'Premier League') ]

#print(df_teams_corners_per_minute.head(5))
#df_teams_corners_per_minute.head()

groups = df_corners_per_team.groupby(['FullName'])

print(groups.head())
groups.describe()
import matplotlib.pyplot as plt
for group in groups:
    #figure()
    #plt.
    print(group[1].FullName.values[0])
    plt.hist(group[1].Minute, bins=45)
    #cumulative=1, density=True, histtype='step'
    axes = plt.gca()
    axes.set_xlim([0,90])
    #axes.set_ylim([0,50])
    plt.show()
   
#df_teams_corners_per_minute['Minute'].hist(by=df_teams_corners_per_minute['count'])

       MatchId ExternalId  Minute  TeamId                 FullName Acronym  \
0          NaN    3631175      15    9847      Paris Saint Germain     PSG   
1          NaN    3631254      44    9847      Paris Saint Germain     PSG   
2          NaN    3631293      51    9847      Paris Saint Germain     PSG   
3          NaN    3631297      52    9837                    Reims     REI   
4          NaN    3631347      73    9847      Paris Saint Germain     PSG   
5          NaN    3631409      90    9847      Paris Saint Germain     PSG   
6          NaN    3633517       3    None                     None    None   
7          NaN    3633530       4    9831                     Nice     NIC   
8          NaN    3633536       5    9831                     Nice     NIC   
9          NaN    3633540       5    9831                     Nice     NIC   
10         NaN    3633542       5    9831                     Nice     NIC   
11         NaN    3633590       9    8639                    Lil

<Figure size 640x480 with 1 Axes>

Almeria


<Figure size 640x480 with 1 Axes>

Amiens


...

<Figure size 640x480 with 1 Axes>

Deportivo Alaves


<Figure size 640x480 with 1 Axes>

Deportivo La Coruna


KeyboardInterrupt: 

In [ ]:
stmt = """SELECT DISTINCT  Matches.Id as MatchId, Goals.ExternalId, convert (bigint,  Goals.Minute) as Minute, Teams.ExternalId as TeamId,
    Teams.FullName, Teams.Acronym,Matches.Season
 FROM [FootballData].[dbo].[Teams]
RIGHT JOIN [Goals] ON Goals.TeamId = Teams.ExternalId
LEFT JOIN [Matches] ON [Matches].ExternalId = Goals.MatchId
AND Matches.Season = '2018/2019'
"""
#WHERE League like 'Premier League'

df_goals_per_team = pd.read_sql(stmt,conn)
#df_goals_per_team = df_goals_per_team['Minute'].notnull()

df_goals_per_team.drop_duplicates(['ExternalId'], inplace=True)
print(df_goals_per_team.shape)
df_goals_per_team.sample(10)

In [ ]:
groups = df_goals_per_team.groupby(['FullName'])

print(groups.head())
groups.describe()
import matplotlib.pyplot as plt
for group in groups:
    #figure()
    #plt.
    print(group[1].FullName.values[0])
    plt.hist(group[1].Minute, bins=45)
    #cumulative=1, density=True, histtype='step'
    axes = plt.gca()
    axes.set_xlim([0,90])
    #axes.set_ylim([0,50])
    plt.show()

In [ ]:
stmt = "SELECT * FROM Corners"
df_corners = pd.read_sql(stmt,conn)
#df_corners = df_corners['Minute'].notnull()
print(df_corners.shape)
df_corners.head()

stmt = "SELECT * FROM Matches"
df_matches = pd.read_sql(stmt,conn)
print(df_matches.shape)
df_matches.head()

result = pd.merge(df_corners.set_index("MatchId"),df_matches.set_index("ExternalId"), how='left')
#result_1718 = result[result.Season == '2017/2018']

#premier_league = result[(result['Season']  == '2017/2018') & (result['League']  == 'Premier League')]
premier_league = result[ (result['Season'] == '2017/2018') & (result['League'] == 'Ligue 1')]

print(result.shape)

#groups = premier_league.groupby(['MatchId'])
#groups.groups.keys()
#df_goals.head()
#groups = df_corners.groupby(['MatchId'])
#groups.groups.keys()
df_goals.head()

minutes_sum = df_corners.groupby(['MatchId', 'TeamId']).apply(
    lambda x: "{%s}" % ', '.join(x.Minute + ' '))
#minutes_sum = df_corners.groupby('MatchId')['MatchId', 'TeamId','Minute']
#print(premier_league[:10].head())
#print(result_premier_league_1718)
premier_league_minutes = premier_league[premier_league['Minute'].notnull()]
print(premier_league_minutes.shape)
minutes_sum
#print(list(premier_league_minutes.columns.values))

In [ ]:
stmt = "SELECT * FROM Goals"
# Excute Query here
df_goals = pd.read_sql(stmt,conn)
print(df_goals.shape)
df_goals.head()


In [ ]:
print(df_matches.shape)
print(df_goals.shape)
goals = pd.merge(df_matches.set_index("ExternalId"), df_goals.set_index("MatchId"),  how='left')
#result = goals[goals['Minute'].notnull()]
print(goals.shape)
#group by season
premier_league = goals[(goals['League'] == 'Premier League') & (goals['Season'] == '2016/2017')]
premier_league.head()

premier_league['Minute'] = premier_league['Minute'].astype(int)
#premier_league.loc[premier_league['Minute']] = premier_league['Minute'].astype(int)

for number in range(0, 90, 10):
    from_minute = number
    to_minute = number + 10
    goals_by_minute = premier_league.groupby(['TeamId'])['Minute'].apply(
        lambda x: len(x.where(
            lambda y : y > from_minute).where(lambda z:z < to_minute).dropna()))   
    #print(goals)
    #with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
       # print(goals)
#goals_teams = goa
#len(teams.groups)
premier_league.head(2)

In [ ]:
result = pd.merge(df_matches.set_index("ExternalId"), df_corners.set_index("MatchId"),  how='left')
#premier_league = result[(result['Season']  == '2017/2018') & (result['League']  == 'Premier League')]
premier_league = result[ (result['Season'] == '2017/2018') & (result['League'] == 'Ligue 1')]
#print(result.shape)


#teams_coreners = df_corners.groupby('TeamId')
goal_matches = pd.merge(df_corners.set_index("MatchId"), df_matches.set_index("ExternalId"), how='right')

#goal_matches_home_team = pd.merge(goal_matches.set_index("HomeTeam_Id"), df_teams.set_index("ExternalId"), how='left')
#goal_matches_home_team.rename(columns={'Ful': 'FullNameHome'}, inplace=True)
#goal_matches_home_team.rename(columns={'Acronym': 'AcronymHome'}, inplace=True)
#goal_matches_home_team.rename(columns={'TeamId': 'HomeTeamId'}, inplace=True)
#goal_matches_home_team_away_team = pd.merge(goal_matches_home_team.set_index("AwayTeam_Id"), df_teams.set_index("ExternalId"), how='left')
#goal_matches_home_team_away_team
#goal_matches_home_team
result.head(2)
#goal_matches_home_team_away_team
#['Minute','TeamId'].apply(
#    lambda x: "{%s}" % ', '.join(x.Minute + ' ' + x.TeamId))
#minutes_sum = result.groupby(['TeamId'], as_index=True)['Minute']hg
#minutes_sum
#['Minute']
#minutes_sum = df_corners.groupby('MatchId')['MatchId', 'TeamId','Minute']
#print(premier_league[:10].head())
#print(result_premier_league_1718)
#premier_league_minutes = premier_league[premier_league['Minute'].notnull()]
#print(premier_league_minutes.shape)
#minutes_sum.first()
#print(list(premier_league_minutes.columns.values))

In [ ]:
goals = pd.merge( df_matches.set_index("ExternalId"), df_goals.set_index("MatchId"), how='left')
#result = goals[goals['Minute'].notnull()]
print(result.shape)
premier_league = goals[(goals['League'] == 'Premier League') & (goals['Season'] == '2016/2017')]
premier_league.head(2)

#premier_league['Minute'] = premier_league['Minute'].astype(int)
#premier_league.loc[premier_league['Minute']] = premier_league['Minute'].astype(int)

#premier_league['TeamId'].hist(by=premier_league['Minute'])

# for number in range(0, 90, 10):
#     from_minute = number
#     to_minute = number + 10
#     goals = premier_league.groupby(['TeamId'])['Minute'].apply(lambda x: len(x.where(lambda y : y > from_minute).where(lambda z:z < to_minute).dropna()))   
#     print(goals)
# #goals_teams = goa
# #len(teams.groups)

#print(teams.groups.keys())

In [ ]:
#groups = df_goals.groupby(['MatchId'])
#groups.groups.keys()
#df_goals.head()
minutes_sum = df_goals.groupby(['MatchId', 'TeamId',], as_index=True, sort=True)
#minutes_sum.last()
minutes_sum.count()
#print(minutes_sum.shape)
#df_goals.head(10)
#groups.groups['1709877']
#minutes_sum.groups
#result = minutes_sum.get_group(('1709702','9748'))

In [ ]:
# def get_last_matches(date, team, x = 10):
#     ''' Get the last x matches of a given team. '''
    
#     #Filter team matches from matches
#     team_matches = df_matches[(df_matches['HomeTeam'] == team) | (df_matches['AwayTeam'] == team)].drop_duplicates(['ExternalId'])
#     team_matches['ExternalId'] = team_matches['ExternalId'].astype(int)
#     #Filter x last matches from team matches
#     last_matches = team_matches[team_matches.Date < date].sort_values(by = 'Date', ascending = False).iloc[0:x,:]

#     #Return last matches
#     return last_matches

# last_matches = get_last_matches("2018-08-27 15:00:00", "Arsenal")
# last_matches.head()